In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
#Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from pylab import rcParams

%matplotlib inline
params = {'axes.labelsize': 14,'axes.titlesize':14, 'text.fontsize': 14, 'legend.fontsize': 14,
          'xtick.labelsize': 13, 'ytick.labelsize': 14}
rcParams['figure.figsize'] = 6.5, 4

matplotlib.rcParams.update(params)


In [ ]:
data = pd.read_csv('Data Mining VU data/training_set_VU_DM_2014.csv')

In [ ]:
df = data.head(500)
df

#  Feature Engineering

1. Extract season: autumn, summer, winter 


In [ ]:
df['date_time']= pd.to_datetime(df['date_time'])
#data.date_time.map(lambda x: x.month)

df['season'] = df.date_time.apply(lambda dt: (dt.month%12 + 3)//3)
df

In [ ]:
# Drop columns with all zeros - only temporary
df = df.dropna(how="all", axis = 1)
df = df.loc[:, (df != 0).any(axis=0)]
df

In [ ]:
# Subselect columns - include booking_book
summary_metrics = pd.DataFrame(df.describe())
summary_metrics.columns

In [ ]:
list(summary_metrics.columns)

In [ ]:
df=df[list(summary_metrics.columns)]

In [ ]:
for col in df.columns:
    print(df[col].dtype)


In [ ]:
summary_metrics

In [ ]:
df.dtypes
df

In [ ]:
df.iloc[:,40:48].

### Calculate correlation and mutual information/information gain

In [ ]:
columns_to_plot=[38,39,42,44,47,48]
g = sns.pairplot( df.iloc[1:500,listed], hue="booking_bool",palette = 'seismic',size=5,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=10),  )



In [ ]:
x=10
0**x

In [ ]:
plt.matshow(df.corr())

